# 协程
- 参考资料
  - http://python.jobbole.com/86481/
  - http://python.jobbole.com/87310/
  - http://segmentfault.com/a/1190000009781688
  
# 迭代器
- 可迭代（Iterable）：直接作用于for循环的变量
- 迭代器（Iterator）：不但可以作用于for循环，还可以被next调用
- list是典型的可迭代对象，但不是迭代器
- 通过isinstance判断
- iterable 和iterator可以转换
  - 通过iter函数
  


In [3]:
# 可迭代
l = [i for i in range(10)]

# l是可迭代的，但不是迭代器
for idx in l:
    print(idx)
    
# range是个迭代器
for i in range(5):
    print(i)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [7]:
# isinstance案例
# 判断某个变量是否是一个实例
# 判断是否可迭代

from collections import Iterable
ll = [1,2,3,4,5]

print(isinstance(ll, Iterable))

from collections import Iterator
print(isinstance(ll, Iterator))

True
False


C:\Users\fukan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  # Remove the CWD from sys.path while we load stuff.


In [12]:
# iter函数

s = 'i love wangxiaojing'

print(isinstance(s, Iterable))
print(isinstance(s, Iterator))

s_iter = iter(s)
print(isinstance(s_iter, Iterable))
print(isinstance(s_iter, Iterator))

True
False
True
True


# 生成器
- generator：一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件：
  - 每次调用都生产出for循环需要的下一个元素
  - 如果达到最后一个后，爆出StopIteration异常
  - 可以被next函数调用
- 如何生成一个生成器
  - 直接使用
  - 如果函数中包括yield，则这个函数就叫生成器
  - next调用函数，遇到yield返回
  

In [14]:
# 直接使用生成器
L = [x*x for x in range(5)] # 放在中括号里是列表生成器
g = (x*x for x in range(5)) # 放在小括号中就是生成器

print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [21]:
# 函数案例

def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3
   
# odd()是调用生成器
g = odd()
one = next(g)
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [24]:
# for循环调用生成器
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        print(b)
        a,b = b, a+b # 注意写法
        n += 1
        
    return 'Done'
fib(5)

1
1
2
3
5


'Done'

In [30]:
# 斐波那契额数列的生成器写法
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        yield b
        a,b = b, a+b # 注意写法
        n += 1
        
    return 'Done'
g = fib(5)

for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


StopIteration: Done

In [31]:
ge = fib(10)
'''
生成器的典型用法是在for中使用
比较常用的典型生成器就是range
'''

for i in ge: # 在for循环中使用生成器
    print(i)

1
1
2
3
5
8
13
21
34
55


# 协程
- 历史历程
  - 3.4引入协程，用yield实现
  - 3.5引入协程语法
  - 实现的协程比较好的包有asyncio，tornado，gevent
- 定义：协程是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或开始执行程序。
- 从技术角度讲，协程就是一个可以暂停执行的函数，或者是生成器
- 协程的实现
  - yield返回
  - send调用
  
- 协程的四个状态

- 协程终止
  - 协程中未处理的异常会向上冒泡，传给next函数或send方法的调用方（即触发协程的对象）
  - 终止协程的一种方式：发送某个哨符值，让协程退出。内置的None和Ellipsis等常量经常用作哨符值==。
  
- yield from
  - 调用协程为了得到返回值，协程必须正常终止
  - 生成器正常终止会发出StopIteration异常，异常对象的value属性保存返回值
  - yield from从内部捕获StopIteration异常
  - 委派生成器
    - 包含yield from表达式的生成器函数
    - 委派生成器在yield from表达式出暂停，调用方法可以直接把数据发给子生成器
    - 子生成器在把产出的值发给调用函数方
    - 子生成器在最后，解释器会抛出StopIteration，并把返回值附加到异常对象上

In [32]:
# 协程代码案例1
def simple_coroutine():
    print('-> start')
    x = yield
    print('-> recived', x)
# 主线程   
sc = simple_coroutine()
print(1111)
# 可以使用sc.send(None)，效果一样
next(sc) # 预激

print(2222)
sc.send('zhexiao')

1111
-> start
2222
-> recived zhexiao


StopIteration: 

In [33]:
# 协程的状态
def simple_coroutine(a):
    print('-> start')
    
    b = yield a
    print('-> recived', a, b)
    
    c = yield a + b
    print('-> recived', a, b, c)
    
# runc
sc = simple_coroutine(5)

aa = next(sc)
print(aa)
bb = sc.send(6) # 5, 6
print(bb)
cc = sc.send(7) # 5, 6, 7
print(cc)

-> start
5
-> recived 5 6
11
-> recived 5 6 7


StopIteration: 

In [34]:
# 案例
def gen():
    for c in 'AB':
        yield c
# list直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from 'AB'
    
print(list(gen_new()))

['A', 'B']
['A', 'B']


In [ ]:
# 委派生成器
